In [2]:
import sys
import pandas as pd
sys.path.append('..')
from examwiz_pkg.examwiz_pkg.gapi_app import grade_book as gb
from examwiz_pkg.examwiz_pkg.gapi_app import mailer as ml
from examwiz_pkg.examwiz_pkg.gapi_app import reporter as rp
from examwiz_pkg.examwiz_pkg.anon_app import anonymizer as an

#### 1. Exam is Created
Admin writes new exam (or is porting current exam into the system)

In [ ]:
#exam2 = gb.create_grade_book('Python Midterm2', in_domain=True)
## Generates new exam that is view only to SupStat Inc users, but can be edited by owner.
exam = '1ZuM7K8ygsAhBjlBr2FE3zyilRwJpT34XLId602OuETE'
print('New Gradebook created. Access at:', gb.get_link(exam))

#Admin links gradebook to a google form and manually structures the questions (needs to be automated)
form_link = 'https://docs.google.com/forms/d/e/1FAIpQLSex4HTuoBouCkkN0XrszdDiqzGVOZwocaTdQKd0zhNxot5SHA/viewform?usp=sf_link'

#### 2. Exam is administered
Cohort students take the exam and provide their solutions to the portal. This is downloadeded from the portal and run through an anonymizer

In [ ]:
gb.fresh_start('demo files')
raw_path = 'demo files/raw_student_exams'
anon_path, key = an.make_anon(raw_path)

#### 3. Send Anonymized Exams to TAs
Randomly split the examinations and send the files to the TAs along with the grading form.

In [4]:
anon_path = 'demo files/anon_student_exams_perm'
key = 'demo files/conversion_key_perm.csv'
form_link = 'https://docs.google.com/forms/d/e/1FAIpQLSdeOgHuin1UD9vCmjezzF0C7rF5h7nUz9RPqLMblcEBlv8i6g/viewform'
exam_id = '1EslwwOrJuor0NyolZTmkQHe6Dc7oN2Ta3TZatuq_Qnc'

In [ ]:
#Find Active TAs
names, emails = ml.get_ta_emails('demo files/TA_data.txt')
num_tas = len(names)
num_exams = len(an.list_files(anon_path))

#Email TAs student files and grading form link, save TA workload for future checking
register = []
for i in range(num_tas):
    exams = an.list_files(anon_path)[(i*num_exams//num_tas):((i+1)*num_exams//num_tas)]
    register.append({'ta': names[i], 'workload': exams, 'email': emails[i]})
    
    em = ml.create_attached_message(
        sender='charles.cohen@nycdatascience.com',
        to=emails[i],
        subject=f'{names[i]}\'s to grade: R Midterm',
        msg=f"""Hi {names[i]},
Here are the R Midterm student exams.
Please submit your grade to them promptly.
Here is the submission form link {form_link}""",
        file_dir = 'demo files/anon_student_exams_perm',
        filenames = exams
    )
    pd.DataFrame(register).to_csv('demo files/ta_workload.csv')

    
   # ml.send_message(em)

#### 4. Check Gradebook at start of day to see who has exams to grade and email them.

[9475, 6376]

In [41]:
miss = [int(i) for i in gb.needs_grading(exam_id, key)]
book = gb.read_grade_book(exam_id)

i = miss[0]
import ast, os
import pandas as pd
wkld = pd.read_csv('demo files/ta_workload_perm.csv')
wkld['workload'] = wkld.apply(lambda l: [os.path.splitext(i)[0] for i in ast.literal_eval(l.workload)], axis=1)
wkld['workload']

0          [6003, 2060, 3070]
1    [8913, 2170, 3655, 6376]
2    [1291, 4260, 3728, 3776]
3    [8331, 6197, 9475, 6826]
Name: workload, dtype: object

#### 4. Admin audits gradebook (manually)

#### 5. Admin flags gradebook for reporting (manually)

#### 6. Generates exam reports and emails to students. (automatic)